> This code accompanies the "[Creating ONNX from scratch II]()" tutorial.

# Block 0: Creating the reference image
In this first block of code we open all the images of the empty container and compute its average using Pillow and Numpy. We will eventually use this "empty-average.JPG" as a reference to recognize whether the container is empty or not.

In [146]:
import numpy as np
from PIL import Image

## Setup:
image_folder = "images/"
empty_containers = [2,3,5,8,9,12,13]
full_containers = [1,4,6,7,10,11,14]
all_containers = empty_containers+full_containers

# Get number of images, image width, and image height:
n_empty = len(empty_containers)
image_width, image_height=Image.open(image_folder + str(empty_containers[0])+".JPG").size


# Create a numpy array of ints to store the average (assume RGB images)
av_arr=np.zeros((image_height,image_width,3),np.int64)

# Build up average pixel intensities, casting each image as an array of ints
for i in empty_containers:
    im_arr=np.array(Image.open(image_folder + str(empty_containers[0])+".JPG"),dtype=np.int64)
    av_arr=av_arr+im_arr/n_empty

# Round values in array and cast as 8-bit integer
av_arr=np.array(np.round(av_arr),dtype=np.uint8)

# Generate and save the average empty container image
out=Image.fromarray(avarr,mode="RGB")
out.save(image_folder+"empty-average.JPG")
out.show()

# Block 1: Generating our ONNX image processing pipeline.
The code below generates our ONNX image processing pipeline. The pipeline is as follows:

- We substract the "empty-average.JPG" from a given image.
- We compute the absolute value of the remaining difference.
- We sum the array of image (absolute) color values in the difference to a single number.
- We check if the number is larger than some treshhold.

The logic of this implementation is simple: compare to the (average) empty image, an image of an empty container should be relatively similar, i.e., its absolute difference should be small. If however the container is full, the image is different, and thus its summed absolute difference compared to the reference image should be large.

In [147]:
# The imports used in this block
from onnx import helper as h
from onnx import TensorProto as tp
from onnx import checker
from onnx import save


# 1. We start by opening the reference image and creating the neccesary ONNX constants:

# The baseline empty container image (average of the 7 empty images)
reference_image=np.array(Image.open(image_folder+"empty-average.JPG"),dtype=np.int64)

# The baseline image as ONNX constant:
c_base = h.make_node('Constant', inputs=[], outputs=['c_base'], name="c_base_node", 
        value=h.make_tensor(name="c_base_value", data_type=tp.INT64, 
        dims=reference_image.shape, 
        vals=reference_image.flatten()))

# The treshold value as ONNX constant; here we select an average of 25 points off (3000000=300*400*25)
image_treshold = numpy.array([3000000]).astype(numpy.int64)
c_cut = h.make_node('Constant', inputs=[], outputs=['c_cut'], name="c_cut_node", 
        value=h.make_tensor(name="c1v", data_type=tp.INT64, 
        dims=image_treshold.shape, 
        vals=image_treshold.flatten()))


# 2. Next, we declare the funtional ONNX nodes in order of appearance:

# Substract input xin from baseline
n1 = h.make_node('Sub', inputs=['xin', 'c_base'], outputs=['min'], name='n1')  

# Compute absolute values of the remaining difference
n2 = h.make_node('Abs', inputs=['min'], outputs=['abs'], name="n2")  

# Sum all the absolute differences
n3 = h.make_node('ReduceSum', inputs=['abs'], outputs=['sum'], name="n3", keepdims=0)  

# See if the sum is less than image_treshold; if it is the image is empty
n4 = h.make_node('Less', inputs=['sum','c_cut'], outputs=['out'], name="n4")  


# 3. Finally, we create the resulting ONNX graph

# Create the graph
g1 = h.make_graph([c_base, c_cut, n1,n2,n3,n4], 'convert_image',
        [h.make_tensor_value_info('xin', tp.INT64, target.shape)],
        [h.make_tensor_value_info('out', tp.BOOL, [1])])

# Create the model and check
m1 = h.make_model(g1, producer_name='scailable-demo')
checker.check_model(m1)

# Save the model
save(m1, 'empty-container.onnx')

# Block 2: Testing
In this block of code we test the created ONNX pipeline using the ONNX runtime. Clearly, all images are classified correctly:

In [148]:
# A few lines to evaluate the stored model, useful for debugging:
import onnxruntime as rt

# Open the model:
sess = rt.InferenceSession("empty-container.onnx")  

# Test all the empty images
print("Iterating through all images")
for i in all_containers:
    
    # Get whether in reality the container is empty
    true_empty = i in empty_containers
    
    # Check image using the ONNX pipeline
    target=numpy.array(Image.open(image_folder + str(i)+".JPG"),dtype=numpy.int64)    
    out = sess.run(["out"], {"xin": target.astype(numpy.int64)})  
    print("Container {} is empty {}, and it is classified as empty {}.".format(i, true_empty, out[0].flatten()))


Iterating through all images
Container 2 is empty True, and it is classified as empty [ True].
Container 3 is empty True, and it is classified as empty [ True].
Container 5 is empty True, and it is classified as empty [ True].
Container 8 is empty True, and it is classified as empty [ True].
Container 9 is empty True, and it is classified as empty [ True].
Container 12 is empty True, and it is classified as empty [ True].
Container 13 is empty True, and it is classified as empty [ True].
Container 1 is empty False, and it is classified as empty [False].
Container 4 is empty False, and it is classified as empty [False].
Container 6 is empty False, and it is classified as empty [False].
Container 7 is empty False, and it is classified as empty [False].
Container 10 is empty False, and it is classified as empty [False].
Container 11 is empty False, and it is classified as empty [False].
Container 14 is empty False, and it is classified as empty [False].


# Continue...
In the [tutorial]() we continue to show how to use the resulting ONNX pipeline and convert it to WebAssembly for fast deployment...